In [1]:
!pwd


/home/notc/Documents/crypto-project


In [2]:
import requests
import pandas as pd 
import warnings
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas_datareader.data as web
from plotly.subplots import make_subplots
import re
warnings.filterwarnings('ignore')

In [3]:
START_DATE = "2021-01-01"
RSI_TIME_WINDOW = 7

In [4]:
# list of urls of historical crypto data and names

urls = ["https://www.cryptodatadownload.com/cdd/Binance_DOGEUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_SOLUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_MATICUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_TRXUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_DOTUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_AVAXUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_LINKUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_UNIUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_XMRUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ETCUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_XLMUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ICPUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_BCHUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_1INCHUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_AAVEUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ALGOUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_ALPACAUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_SUSHIUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_FILUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_HBARUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_QNTUSDT_d.csv",
        "https://www.cryptodatadownload.com/cdd/Binance_FTMUSDT_d.csv"
       ]
    
crypto_names = [
        "TUSD (TUSD)",
        "1inch (1INCH)",
        "Aave (AAVE)",
        "Cardano (ADA)",
        "Algorand (ALGO)",
        "Alpaca Finance (ALPACA)",
        "USD Coin (USDC)",
        "Binance Coin (BNB)",
        "Dogecoin (DOGE)",
        "XRP (XRP)",
        "Solana (SOL)",
        "Polygon (MATIC)",
        "Litecoin (LTC)",
        "TRON (TRX)",
        "Dogecoin (DOGE)",
        "SushiSwap (SUSHI)",
        "Avalanche (AVAX)",
        "USDT/DAI (USDTDAI)",
        "Chainlink (LINK)",
        "Uniswap (UNI)",
        "XMR (XMR)",
        "ETC (ETC)",
        "XLM (XLM)",
        "Internet Computer (ICP)",
        "Bitcoin Cash (BCH)"
]

In [5]:

def df_loader(urls, start_date="2019-07-01"):
    """
    Loads cryptocurrency price data from a list of URLs and returns a DataFrame and a list of filenames.

    Args:
    urls: A list of URLs pointing to CSV files containing cryptocurrency price data.
    start_date: A string representing the start date of the data.

    Returns:
    A tuple of a DataFrame and a list of filenames.
    """

    filenames = []
    all_df = pd.DataFrame()
    for url in urls:
        try:
            req = requests.get(url, verify=False)
            url_content = req.content
            pattern = r'_[A-Z]+(?=USDT)'
            match = re.search(pattern, url)
            if match:
                ticker_symbol = match.group(0)[1:]
                filenames.append(ticker_symbol)
                csv_file = open(ticker_symbol + "USDT_d.csv", 'wb')
                csv_file.write(url_content)
                csv_file.close()

                df = pd.read_csv(ticker_symbol + "USDT_d.csv", header=1, parse_dates=["Date"])
                df = df[df["Date"] > start_date]
                df.index = df["Date"]
                df.drop(labels=[df.columns[0], df.columns[1], df.columns[7]], axis=1, inplace=True)
                # df.drop(labels=["Volume 1INCH"], axis=1, inplace=True)
                all_df = pd.concat([all_df, df], ignore_index=False)
        except FileNotFoundError:
            pass
    return all_df, filenames


In [6]:
all_df, filenames = df_loader(urls, start_date=START_DATE)
crypto_df = []


In [7]:
def computeRSI(data, time_window):
    """
    Computes the Relative Strength Index (RSI) for a given time series.

    Args:
    data: A pandas DataFrame containing the time series data.
    time_window: The number of periods to use for the RSI calculation.

    Returns:
    A pandas Series containing the RSI values.
    """

    diff = data.diff(1)
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)

    return rsi


In [8]:
def computeMovingAverage(data, time_window):
    """
    Computes the moving average for a given time series.

    Args:
    data: A pandas DataFrame containing the time series data.
    time_window: The number of periods to use for the moving average calculation.

    Returns:
    A pandas Series containing the moving average values.
    """

    moving_average = data.rolling(window=time_window).mean()
    return moving_average


In [9]:
def computeBollingerBands(data, time_window, num_stds):
    """
    Computes Bollinger Bands for a given time series.

    Args:
    data: A pandas DataFrame containing the time series data.
    time_window: The number of periods to use for the moving average calculation.
    num_stds: The number of standard deviations to use for the Bollinger Bands calculation.

    Returns:
    A pandas DataFrame containing the Bollinger Bands values.
    """

    moving_average = data.rolling(window=time_window).mean()
    standard_deviation = data.rolling(window=time_window).std()
    BOLU = moving_average + num_stds * standard_deviation
    BOLD = moving_average - num_stds * standard_deviation

    return pd.DataFrame({"Standard Deviation": standard_deviation, "Moving Average": moving_average, "Upper Band": BOLU, "Lower Band": BOLD})


In [10]:
# 50 period simple moving average
MA50 = computeMovingAverage(all_df["Close"], 50)
all_df["MA50"] = MA50

# 20 period Bollinger Bands
bb = computeBollingerBands(all_df["Close"], 20, 2)

STD20 = bb["Standard Deviation"]
MA20 = bb["Moving Average"]
BOLU = bb["Upper Band"]
BOLD = bb["Lower Band"]

all_df["STD20"] = STD20
all_df["MA20"] = MA20
all_df["BOLU"] = BOLU
all_df["BOLD"] = BOLD

#RSI
rsi = computeRSI(all_df["Close"], 7)
all_df["RSI"] = rsi
all_df


,Symbol,Open,High,Low,Close,Volume USDT,tradecount,MA50,STD20,MA20,BOLU,BOLD,RSI
Date,,,,,,,,,,,,,
2023-05-19,DOGEUSDT,0.07344,0.07406,0.07300,0.07350,2.470914e+07,59222,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-18,DOGEUSDT,0.07484,0.07492,0.07161,0.07344,5.838036e+07,135432,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-17,DOGEUSDT,0.07276,0.07547,0.07228,0.07483,5.628054e+07,133875,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-16,DOGEUSDT,0.07190,0.07325,0.07087,0.07275,2.569240e+07,73596,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-15,DOGEUSDT,0.07195,0.07368,0.07084,0.07191,3.635860e+07,91791,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-06,FTMUSDT,0.02060,0.02450,0.01965,0.02442,4.241330e+06,28270,0.128611,0.009426,0.029692,0.048543,0.010841,30.403344
2021-01-05,FTMUSDT,0.02050,0.02144,0.01867,0.02063,4.297358e+06,26541,0.116032,0.007983,0.028111,0.044076,0.012145,24.398229
2021-01-04,FTMUSDT,0.01892,0.02082,0.01651,0.02047,4.659475e+06,32673,0.107494,0.004400,0.026284,0.035084,0.017484,24.163167
